In [1065]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1066]:
%cd /content/drive/MyDrive/ASP Grp 3 Panda 冠军队 Team/ASP AIML Model Training

/content/drive/MyDrive/ASP Grp 3 Panda 冠军队 Team/ASP AIML Model Training


In [1067]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1524]:
import json
import tarfile
import requests

import random

import pandas as pd
import numpy as np

import torch
from transformers import LongformerForQuestionAnswering
from transformers import LongformerTokenizer
from transformers import LongformerConfig


import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import preprocessing

In [1538]:
with open("intents.json", "r") as f:
  intents = json.load(f)

print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'How are you?', 'Hi Chatbot', 'Hello', 'Good morning', 'Good afternoon', 'Good evening', 'Howdy chatbot', 'Are you the Question and Answer chatbot I am looking for?'], 'responses': ['Hi there, is there any question you have that I can answer?', 'Hello, I am here to answer to answer any questions you possibly have. Do not hesistate to ask me any as it is my duty to help you/']}, {'tag': 'farewell', 'patterns': ['Bye', 'See you later', 'Goodbye', 'See you next time', 'I will be back later', 'Hope to see you again', 'Good night'], 'responses': ['Thank you. Do not hesistate to come back to ask me anymore questions', 'Thank you. Please ask me more questions later if you have more to ask.']}, {'tag': 'about', 'patterns': ['What is Chronic Kidney Disease?', 'Is Chronic Kidney Disease the same as CKD?', 'How does CKD affect the kidneys?'], 'responses': ['Chronic Kidney Disease also known as CKD is a condition in which the kidneys are d

In [1539]:
random.seed(42)

all_response = []
all_pattern = []
all_tag = []
tags = []
for intent in intents["intents"]:
  tag = intent["tag"]
  if tag == "greeting" or tag == "farewell":
    continue
  tags.append(tag)
  for pattern in intent["patterns"]:
    rand_response = random.choice(intent["responses"])
    all_response.append(rand_response)
    all_pattern.append(pattern)
    all_tag.append(tag)

all_response, all_pattern, all_tag

(['Chronic Kidney Disease also known as CKD is a condition in which the kidneys are damaged and are not able to filter waste and excess fluids from the body as effectively as they should. It is a progressive disease, which means that it gets worse over time.',
  'Chronic Kidney Disease also known as CKD is a condition in which the kidneys are damaged and are not able to filter waste and excess fluids from the body as effectively as they should. It is a progressive disease, which means that it gets worse over time.',
  'Chronic Kidney Disease also known as CKD is a condition in which the kidneys are damaged and are not able to filter waste and excess fluids from the body as effectively as they should. It is a progressive disease, which means that it gets worse over time.',
  'There are several causes of CKD, including diabetes, high blood pressure, and certain infections.',
  'There are several causes of CKD, including diabetes, high blood pressure, and certain infections.',
  'There ar

In [1997]:
#most of the description are extracted from https://en.wikipedia.org/wiki/Chronic_kidney_disease (in this case reliable although it is open-source as there are many citations) and ChatGPT

description = '''CKD is the same as Chronic Kidney Disease. 
Chronic Kidney Disease (CKD) is a type of kidney disease in which a gradual loss of kidney function occurs over a period of months to years. 
Chronic kidney disease also affects the kidneys by damaging them and hence are not able to filter waste and excess fluids from the body as effectively as they should. 
CKD is a progressive and hereditary disease.
High blood pressure, diabetes, and certain infections are causes of CKD.
In 2015, it caused 1.2 million deaths, up from 409,000 in 1990. 
Chronic kidney disease affected 753 million people globally in 2016. 
The causes that contribute to the greatest number of deaths are high blood pressure at 550,000, followed by diabetes at 418,000, and glomerulonephritis at 238,000. 
High blood pressure is one of the causes of CKD. 
Diabetes is one of the causes of CKD. 
Glomerulonephritis is one of the causes of CKD. 
About one of five adults with hypertension and one of three adults with diabetes have CKD. 
CKD increases the risk of cardiovascular disease, and people with CKD often have other risk factors for heart disease, such as high blood lipids. 
Chronic kidney disease results in worse all-cause mortality (the overall death rate) which increases as kidney function decreases. 
The leading cause of death in chronic kidney disease is cardiovascular disease, regardless of whether there is progression to stage 5. 
However, it is associated with an increased short-term mortality due to complications of the surgery. 
There is currently no cure for CKD. 
Initially generally no symptoms are seen. 
Later on symptoms of CKD can be swelling in the ankles and feet, fatigue, changes in urine output, anemia, bone disease, high blood pressure, infections, nerve damage and poor nutrition. Worst cases include kidney failure.
In some cases, people with a family history of CKD have a higher chance of developing the disease themselves. 
To prevent people that have a family history of CKD from developing CKD, they can maintain a healthy lifestyle, manage underlying health conditions, and get regular check-ups to monitor their kidney function.
CKD can be inherited. There are several genetic conditions that can cause CKD, such as polycystic kidney disease and Alport syndrome. However, in many cases, CKD is not inherited and is caused by other factors such as diabetes, high blood pressure, and certain infections. 
The outlook for someone with CKD depends on the severity of the disease, the underlying cause, and the patient's overall health. 
Maintaining a healthy blood pressure and good blood sugar control through lifestyle changes such as eating healthily, getting regular exercise, and taking medications as prescribed can help prevent or slow the progression of CKD. 
It is possible to still have CKD after being diagnosed if no early treatment is performed.
It is generally not possible to donate a kidney if you have CKD. Donors must be in good health with normal kidney function, and CKD is generally considered to be a disqualifying condition. 
Fatigue is one of the symptoms of CKD.
Anemia is a symptom of CKD.
Changes in urine output is a symptom of CKD.
CKD affect the body with symptoms such as swelling in the ankles and feet, fatigue, changes in urine output, anemia, bone disease, high blood pressure, infections, nerve damage and poor nutrition. Worst cases include kidney failure.
Complications associated with CKD can relate to hormonal dysfunction of the kidneys and include (in chronological order) high blood pressure, bone disease, and anemia. 
Additionally CKD patients have markedly increased cardiovascular complications with increased risks of death and hospitalization. 
Causes of chronic kidney disease include diabetes, high blood pressure, glomerulonephritis, and polycystic kidney disease. 
High blood pressure is a common cause of CKD. 
Diabetes is a common cause of CKD. 
Diabetes contribute to having CKD. 
High blood pressure contribute to having CKD.  
CKD can be prevented or slowed by managing underlying conditions such as diabetes and high blood pressure, and by maintaining a healthy lifestyle.
The relationship between CKD and diabetes is that that the person has a higher chance of developing CKD if the person has diabetes. 
The relationship between CKD and high blood pressure is that the person has a higher chance of developing CKD if the person has high blood pressure. 
CKD can be reversed in the early stages
CKD is impossible to reverse in the late stages. 
CKD can be prevented or slowed by managing underlying conditions such as diabetes and high blood pressure, and by maintaining a healthy lifestyle. 
CKD can be treated with medication. 
CKD is treated by managing the underlying cause of the disease and slowing the progression of the kidney damage in adults and children.  
CKD is treated with the involvement of medications, lifestyle changes (such as eating healthily and getting regular exercise), and in advanced stages, with dialysis or a kidney transplant. 
To manage the underlying cause of the disease and slowing the progression of the kidney damage in adults and children, the treatment may involve medications, lifestyle changes (such as eating healthily and getting regular exercise), and in advanced stages, with dialysis or a kidney transplant. 
CKD be managed with diet and exercise most of the time alone in the early stages. 
People with CKD that have high blood pressure can be treated with angiotensin converting enzyme inhibitors (ACE inhibitors) or angiotensin receptor blockers (ARBs) (medications).
People with CKD that have diabetes can be treated with insulin or oral diabetes medications.
CKD can cause high cholesterol, which can increase the risk of heart disease. Statins or fibrates (medications) may be used to treat high cholesterol in people with CKD. 
CKD can cause skin problems. 
People with CKD can use emollients or antihistamines (medications) to treat skin problems. 
CKD can cause poor nutrition, which can lead to a range of complications such as anemia and muscle weakness. Vitamin and mineral supplements (medications) may be used to treat poor nutrition in people with CKD. 
CKD can cause nerve damage. Calcium channel blockers or alpha-lipoic acid (medications) may be used to treat nerve damage in people with CKD. 
Swelling in the ankles and feet is one of the symptoms of CKD. 
Diuretics or ACE inhibitors (medications) can be used to treat swelling or urine output changes in people with CKD.
Bisphosphonates or calcimimetics (medications) can be used to treat bone disease in people with CKD. 
Erythropoietin or iron supplements (medications) can be used to treat anemia in people with CKD. 
Erythropoietin or iron supplements (medications) can be used to treat fatigue in people with CKD. 
CKD can be treated with antibotics (infections that occur due to it).
CKD can be treated with a special diet, dietary changes and exercise.
The fact that that CKD can be treated with gene cell therapy is uncertain as treatment is still under research. 
The fact that that CKD can be treated with stem cell threapy is uncertain as treatment is still under research. 
CKD can be treated with a kidney transplant. 
CKD can be treated with dialysis in the later stage of CKD.
CKD can be treated with exercise in the early stages.
CKD is treated or prevented in the early stages by maintaining a healthy lifestyle, healthy diet and sometimes medication and in the late stages, by having a kidney transplant.
Natural remedies, herbal supplements, acupuncutre, supplements, or home remedies can be used as part of the treatment plan or considered to be treatment for CKD but it is advisable to consult a healthcare provider first. 
There is limited evidence to support the usage of natural remedies as treatment for CKD. 
There is limited evidence to support the usage of herbal supplements as treatment for CKD. 
There is limited evidence to support the usage of acupuncutre as treatment for CKD. 
There is limited evidence to support the usage of supplements as treatment for CKD. 
There is limited evidence to support the usage of home remedies as treatment for CKD. 
CKD is diagnosed and monitored through blood, and urine tests, and imaging tests such as ultrasound or CT scan. To conduct the tests, one has to travel to a polyclinic or hospital which can be quite inconvenient. The app and hardware diagnose CKD by checking ammonia level based on user's breath. After checking, the user can see their CKD stage and advices on what to do next will be given by the chatbot. User can detect their CKD stages through the app.
Screening at-risk people (with hypertension, history of cardiovascular disease, diabetes, obesity, aged > 60 years, subjects with African American ancestry, those with a history of kidney disease in the past, and subjects who have relatives who had kidney disease requiring dialysis.) is recommended. Initial treatments may include medications to lower blood pressure, blood sugar, and cholesterol. 
Screening those who have neither symptoms nor risk factors for CKD is not recommended. 
Kidney ultrasonography is useful for diagnostic and prognostic purposes in chronic kidney disease. 
Other recommendations to prevent CKD from worsening include staying active, and certain dietary changes such as a low-salt diet and the right amount of protein. 
At latter stages of CKD, people might require hemodialysis, peritoneal dialysis, or a kidney transplant for survival. 
To prevent CKD, maintaining a healthy blood pressure and good blood sugar control through lifestyle changes such as eating healthily, getting regular exercise, and taking medications as prescribed can help.
Phosphate binders are used to control the serum phosphate levels, which are usually elevated in advanced chronic kidney disease. 
Phosphodiesterase-5 inhibitors and zinc may improve sexual dysfunction in men. 
Aggressive blood pressure lowering decreases people's risk of death. 
Aggressive treatment of high blood lipids is recommended. 
A low-protein, low-salt diet may result in slower progression of CKD and reduction in proteinuria as well as controlling symptoms of advanced CKD to delay dialysis start. 
A tailored low-protein diet, designed for low acidity, may help prevent damage to kidneys for people with CKD. 
Additionally, controlling salt ingestion helps to decrease the incidence of coronary heart disease, lowering blood pressure and reducing albuminuria. 
Calcitriol is recommended for vitamin D deficiency and control of metabolic bone disease. 
Obesity may have a negative impact in CKD, increasing the risk of disease progression to ESKD or kidney failure compared to controls with healthy weight, and when in advanced stages also may hinder people's eligibility to kidney transplantation. 
Having a healthy lifestyle can slow down and to some extent prevent CKD.
Lifestyle interventions may provide some health benefits, namely improving body weight, low density lipoprotein (LDL) cholesterol and diastolic blood pressure (DBP), when compared to usual care or controls. 
Lifestyle changes that manages CKD includes eating food that is low in salt, sugar, and fat and more fruits, vegetables, and whole grains.
People with CKD experience sleep disorders, thus not being able to get quality sleep. There are several strategies that could help, such as relaxation techniques, exercise, acupressure and medication: 
Weak evidence demonstrates that exercise may be helpful with sleep regulation. Nevertheless, exercise possibly decreases fatigue and depression in people with CKD. 
Acupressure may have slight effects on latency and sleep duration, as well as on fatigue reduction. 
A conclusion cannot be drawn to ensure which is the best guidance to improve sleep quality. 
There is currently limited evidence suggesting that eHealth interventions may improve dietary sodium intake and fluid management for people with chronic kidney disease (CKD). 
Most countries agree that nephrology referral is required by Stage 4 CKD. 
It may also be useful at an Stage 3 and earlier, when blood pressure is difficult to control, or when hematuria or other findings suggest either a primarily glomerular disorder or secondary disease amenable to specific treatment. 
At Stage 5 CKD, Kidney replacement therapy is usually required, in the form of either dialysis or a kidney transplant.
High-intensity home hemodialysis seems to be associated with improved survival and a greater quality of life. 
People with Stage 5 CKD are at increased overall risk for cancer. This risk is particularly high in younger people and gradually diminishes with age. 
In children, growth failure is a common complication from CKD. 
Children with CKD will be shorter than 97% of children the same age and sex.
CKD affects both men and women. However, some studies have suggested that it may be more common in men, particularly in older men. 
African, Hispanics, and South Asians, particularly those from Pakistan, Sri Lanka, Bangladesh, and India, are at high risk of developing CKD. 
Africans are at greater risk due to the number of people affected with hypertension among them. 
As an example, 37% of ESKD cases in African Americans can be attributed to high blood pressure, compared with 19% among Caucasians. 
Treatment efficacy also differs between racial groups. 
Administration of antihypertensive drugs generally halts disease progression in white populations but has little effect in slowing kidney disease among black people, and additional treatment such as bicarbonate therapy is often required. 
While lower socioeconomic status contributes to the number of people affected with CKD, differences in the number of people affected by CKD are still evident between Africans and Whites when controlling for environmental factors. 
'''

description = description.replace("\n", "")
description, len(description)

("CKD is the same as Chronic Kidney Disease. Chronic Kidney Disease (CKD) is a type of kidney disease in which a gradual loss of kidney function occurs over a period of months to years. Chronic kidney disease also affects the kidneys by damaging them and hence are not able to filter waste and excess fluids from the body as effectively as they should. CKD is a progressive and hereditary disease.High blood pressure, diabetes, and certain infections are causes of CKD.In 2015, it caused 1.2 million deaths, up from 409,000 in 1990. Chronic kidney disease affected 753 million people globally in 2016. The causes that contribute to the greatest number of deaths are high blood pressure at 550,000, followed by diabetes at 418,000, and glomerulonephritis at 238,000. High blood pressure is one of the causes of CKD. Diabetes is one of the causes of CKD. Glomerulonephritis is one of the causes of CKD. About one of five adults with hypertension and one of three adults with diabetes have CKD. CKD incr

In [1998]:
#prepare description (context) for all questions and answers
all_description = [description] * len(all_pattern)
len(all_description)

101

In [1999]:
df = pd.DataFrame({
                    "Intent": all_tag,
                    "Context": all_description,
                    "Question": all_pattern,
                    "Response": all_response
                  })
df

,Intent,Context,Question,Response
0,about,CKD is the same as Chronic Kidney Disease. Chr...,What is Chronic Kidney Disease?,Chronic Kidney Disease also known as CKD is a ...
1,about,CKD is the same as Chronic Kidney Disease. Chr...,Is Chronic Kidney Disease the same as CKD?,Chronic Kidney Disease also known as CKD is a ...
2,about,CKD is the same as Chronic Kidney Disease. Chr...,How does CKD affect the kidneys?,Chronic Kidney Disease also known as CKD is a ...
3,causes,CKD is the same as Chronic Kidney Disease. Chr...,What are the causes of CKD?,"There are several causes of CKD, including dia..."
4,causes,CKD is the same as Chronic Kidney Disease. Chr...,Is diabetes one of the causes of CKD?,"There are several causes of CKD, including dia..."
...,...,...,...,...
96,race_common,CKD is the same as Chronic Kidney Disease. Chr...,Is CKD more common in certain racial or ethnic...,Some racial and ethnic groups may be at an inc...
97,race_common,CKD is the same as Chronic Kidney Disease. Chr...,For which race or ethnic group is CKD more com...,Some racial and ethnic groups may be at an inc...
98,special_diet,CKD is the same as Chronic Kidney Disease. Chr...,Can CKD be treated with a special diet?,Making dietary changes may be an important par...
99,special_diet,CKD is the same as Chronic Kidney Disease. Chr...,Can special diet be used to treat CKD?,Making dietary changes may be an important par...


In [2000]:
#Details about the pretrained model: https://metatext.io/models/valhalla-longformer-base-4096-finetuned-squadv1

model = LongformerForQuestionAnswering.from_pretrained("valhalla/longformer-base-4096-finetuned-squadv1")
tokenizer = LongformerTokenizer.from_pretrained("valhalla/longformer-base-4096-finetuned-squadv1")

In [2013]:
#prepare random question and answer to see how does it look like when tokenized

rand_num = np.random.randint(0,len(df))

question = df["Question"][rand_num]

context = df["Context"][rand_num]

actual_answer = df["Response"][rand_num]

question, actual_answer

('Can CKD be treated with medications for poor nutrition?',
 'CKD can cause poor nutrition, which can lead to a range of complications such as anemia and muscle weakness. Medications such as vitamin and mineral supplements may be used to treat poor nutrition in people with CKD. However, these medications may not be suitable for everyone and may cause side effects.')

In [2014]:
#check if is running gpu
#set the model to be using gpu if it is being used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device currently using is",device)
model = model.to(device)

Device currently using is cpu


In [2015]:
#get the ids for the question and answers based on the LongformerTokenizer
encoding = tokenizer.encode_plus(question, context, return_tensors = "pt")
encoded_input_ids = encoding["input_ids"]
print(f"There are {len(encoded_input_ids[0])} ids")

There are 2810 ids


In [2016]:
#inspect the tokenized encoded_input
# tokens = tokenizer.convert_ids_to_tokens(encoded_input_ids[0].tolist())
# for token, id in zip(tokens, encoded_input_ids[0].tolist()):
#     print(f"Token: {token:30} ID: {id}")

In [2017]:
# default is local attention (focused on subsets) everywhere
# the forward method will automatically set global attention (focused as a whole) on question tokens
attention_mask = encoding["attention_mask"]

In [2018]:
#get the start and end logits after feeding the input ids and attention mask into the Longformer model
start_logits, end_logits = model(encoded_input_ids, attention_mask = attention_mask).values()

#get all tokens so that we can get the reference of which part the answer starts and ends
all_tokens = tokenizer.convert_ids_to_tokens(encoded_input_ids[0].tolist())

In [2019]:
#evaluate model's performance
#as long as the answers make sense we will treat it as correct even though it might not match the actual answer

start_answer = torch.argmax(start_logits)
end_answer = torch.argmax(end_logits)
if end_answer > start_answer:
    #get the raw answer based on all_tokens
    answer_tokens = all_tokens[start_answer: end_answer+1]

    #decode to remove special g character. Special g character is used to as a way to substitute spacing so that more tokens can be used
    answer = tokenizer.decode(tokenizer.convert_tokens_to_ids(answer_tokens)) + "\nIf this is not what you are looking for, please rephrase your question."
    if answer.startswith("<s>"):
      answer = "I do not understand this question. Can you rephrase or ask another question?"
else:
    answer = "I do not understand this question. Can you rephrase or ask another question?"
    
print(f"Question:\n{question}")
print(f"\nAnswer:\n{answer.strip().capitalize()}")
print(f"\nActual Answer:\n{actual_answer.strip().capitalize()}")

Question:
Can CKD be treated with medications for poor nutrition?

Answer:
Can lead to a range of complications such as anemia and muscle weakness
if this is not what you are looking for, please rephrase your question.

Actual Answer:
Ckd can cause poor nutrition, which can lead to a range of complications such as anemia and muscle weakness. medications such as vitamin and mineral supplements may be used to treat poor nutrition in people with ckd. however, these medications may not be suitable for everyone and may cause side effects.


In [805]:
#create a function to prepare the model for Q&A 
def longformerQandA(question, context):
    encoding = tokenizer.encode_plus(question, context, return_tensors="pt")
    encoded_input_ids = encoding["input_ids"]

    # default is local attention everywhere
    # the forward method will automatically set global attention on question tokens
    attention_mask = encoding["attention_mask"]

    start_logits, end_logits = model(encoded_input_ids, attention_mask=attention_mask).values()
    all_tokens = tokenizer.convert_ids_to_tokens(encoded_input_ids[0].tolist())

    start_answer = torch.argmax(start_logits)
    end_answer = torch.argmax(end_logits)
    if end_answer >= start_answer:
        answer_tokens = all_tokens[start_answer:end_answer+1]
        answer = tokenizer.decode(tokenizer.convert_tokens_to_ids(answer_tokens))
        
    else:
        return "I do not understand this question. Can you rephrase or ask another question?"
        
    return answer

In [512]:
question = input("Please ask me a question:\n")
invalid = False
while True:
  if not invalid:
    print(f"Answer:\n{longformerQandA(question, context)}")
    quit = input("\nDo you want to ask me another question?(Y/N):")
    if quit in ['N', 'n']:
      print("Goodbye")
      break
    elif quit in ['Y', 'y']:
      question = input("\nPlease ask me a question:\n")
    else:
      print("invalid option. Please choose Y (Yes) or N (no).")
      invalid = True
  else:
    quit = input("\nDo you want to ask me another question?(Y/N):")H
    question = input("\nPlease ask me a question:\n")
    invalid = False

Please ask me a question:
Is anemia a symptom of CKD
Answer:
low red blood cell count

Do you want to ask me another question?(Y/N):a
invalid option. Please choose Y (Yes) or N (no).

Do you want to ask me another question?(Y/N):Is anemia a symptom of CKD

Please ask me a question:
Is anemia a symptom of CKD
Answer:
low red blood cell count

Do you want to ask me another question?(Y/N):Is hypertension a symptom of CKD
invalid option. Please choose Y (Yes) or N (no).

Do you want to ask me another question?(Y/N):y

Please ask me a question:
Is hypertension a symptom of CKD
Answer:
I do not understand this question. Can you rephrase or ask another question?

Do you want to ask me another question?(Y/N):Is high blood pressure a symptom of CKD
invalid option. Please choose Y (Yes) or N (no).

Do you want to ask me another question?(Y/N):y

Please ask me a question:
Is high blood pressure a symptom of CKD
Answer:
 High blood pressure is a common cause of CKD

Do you want to ask me another q

KeyboardInterrupt: ignored